# End-to-End Trading Project
## Часть 2: Feature Engineering и EDA

### В этом ноутбуке:

1. **Загрузка данных** из предыдущего ноутбука
2. **Технические индикаторы** - SMA, EMA, RSI, MACD, Bollinger Bands, ATR
3. **Лаговые признаки** - прошлые значения для предсказания
4. **Целевые переменные** - что будем предсказывать
5. **EDA** - анализ признаков и их связей
6. **Подготовка данных** для моделирования

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import json
import os
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

print('Библиотеки загружены')

In [ ]:
# Загружаем данные
data_dir = 'data'
df = pd.read_parquet(f'{data_dir}/market_data.parquet')

with open(f'{data_dir}/metadata.json', 'r') as f:
    metadata = json.load(f)

print(f'Загружено записей: {len(df):,}')
print(f'Период: {df["date"].min().date()} - {df["date"].max().date()}')
print(f'Акций: {df["ticker"].nunique()}')
print(f'\nКолонки: {df.columns.tolist()}')

## 1. Технические Индикаторы

Создаём набор технических индикаторов, которые трейдеры используют для анализа:

### Трендовые индикаторы:
- **SMA** (Simple Moving Average) - простая скользящая средняя
- **EMA** (Exponential Moving Average) - экспоненциальная скользящая средняя

### Моментум индикаторы:
- **RSI** (Relative Strength Index) - индекс относительной силы
- **MACD** - схождение/расхождение скользящих средних
- **ROC** (Rate of Change) - скорость изменения

### Волатильность:
- **Bollinger Bands** - полосы Боллинджера
- **ATR** (Average True Range) - средний истинный диапазон

### Объёмные индикаторы:
- **OBV** (On-Balance Volume) - балансовый объём
- **VWAP** - средневзвешенная по объёму цена

In [ ]:
def add_technical_indicators(df):
    """
    Добавляет полный набор технических индикаторов.
    Обрабатывает каждую акцию отдельно.
    """
    result_dfs = []
    
    for ticker in df['ticker'].unique():
        ticker_df = df[df['ticker'] == ticker].copy().sort_values('date')
        
        close = ticker_df['close']
        high = ticker_df['high']
        low = ticker_df['low']
        volume = ticker_df['volume']
        
        # === Трендовые индикаторы ===
        
        # SMA
        ticker_df['sma_5'] = close.rolling(5).mean()
        ticker_df['sma_10'] = close.rolling(10).mean()
        ticker_df['sma_20'] = close.rolling(20).mean()
        ticker_df['sma_50'] = close.rolling(50).mean()
        ticker_df['sma_200'] = close.rolling(200).mean()
        
        # EMA
        ticker_df['ema_12'] = close.ewm(span=12).mean()
        ticker_df['ema_26'] = close.ewm(span=26).mean()
        ticker_df['ema_50'] = close.ewm(span=50).mean()
        
        # Позиция цены относительно SMA
        ticker_df['price_sma20_ratio'] = close / ticker_df['sma_20']
        ticker_df['price_sma50_ratio'] = close / ticker_df['sma_50']
        
        # Golden/Death Cross signals
        ticker_df['sma_cross'] = (ticker_df['sma_50'] > ticker_df['sma_200']).astype(int)
        
        # === Моментум индикаторы ===
        
        # RSI
        delta = close.diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / (loss + 1e-10)
        ticker_df['rsi'] = 100 - (100 / (1 + rs))
        
        # MACD
        ticker_df['macd'] = ticker_df['ema_12'] - ticker_df['ema_26']
        ticker_df['macd_signal'] = ticker_df['macd'].ewm(span=9).mean()
        ticker_df['macd_hist'] = ticker_df['macd'] - ticker_df['macd_signal']
        
        # ROC (Rate of Change)
        ticker_df['roc_5'] = close.pct_change(5) * 100
        ticker_df['roc_10'] = close.pct_change(10) * 100
        ticker_df['roc_20'] = close.pct_change(20) * 100
        
        # Momentum
        ticker_df['momentum_10'] = close - close.shift(10)
        
        # === Волатильность ===
        
        # Bollinger Bands
        bb_sma = close.rolling(20).mean()
        bb_std = close.rolling(20).std()
        ticker_df['bb_upper'] = bb_sma + 2 * bb_std
        ticker_df['bb_lower'] = bb_sma - 2 * bb_std
        ticker_df['bb_width'] = (ticker_df['bb_upper'] - ticker_df['bb_lower']) / bb_sma
        ticker_df['bb_position'] = (close - ticker_df['bb_lower']) / (ticker_df['bb_upper'] - ticker_df['bb_lower'] + 1e-10)
        
        # ATR
        tr1 = high - low
        tr2 = abs(high - close.shift())
        tr3 = abs(low - close.shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        ticker_df['atr'] = tr.rolling(14).mean()
        ticker_df['atr_percent'] = ticker_df['atr'] / close * 100
        
        # Historical Volatility
        ticker_df['volatility_10'] = close.pct_change().rolling(10).std() * np.sqrt(252)
        ticker_df['volatility_20'] = close.pct_change().rolling(20).std() * np.sqrt(252)
        
        # === Объёмные индикаторы ===
        
        # Volume SMA
        ticker_df['volume_sma_20'] = volume.rolling(20).mean()
        ticker_df['volume_ratio'] = volume / (ticker_df['volume_sma_20'] + 1e-10)
        
        # OBV (On-Balance Volume)
        obv = [0]
        for i in range(1, len(close)):
            if close.iloc[i] > close.iloc[i-1]:
                obv.append(obv[-1] + volume.iloc[i])
            elif close.iloc[i] < close.iloc[i-1]:
                obv.append(obv[-1] - volume.iloc[i])
            else:
                obv.append(obv[-1])
        ticker_df['obv'] = obv
        ticker_df['obv_sma'] = ticker_df['obv'].rolling(20).mean()
        
        # === Ценовые паттерны ===
        
        # Свечные паттерны (упрощённо)
        ticker_df['body'] = close - ticker_df['open']
        ticker_df['body_percent'] = ticker_df['body'] / ticker_df['open'] * 100
        ticker_df['upper_shadow'] = high - pd.concat([close, ticker_df['open']], axis=1).max(axis=1)
        ticker_df['lower_shadow'] = pd.concat([close, ticker_df['open']], axis=1).min(axis=1) - low
        
        # Daily range
        ticker_df['daily_range'] = (high - low) / low * 100
        
        # Gap
        ticker_df['gap'] = (ticker_df['open'] - close.shift()) / close.shift() * 100
        
        result_dfs.append(ticker_df)
    
    return pd.concat(result_dfs, ignore_index=True)

# Применяем
df = add_technical_indicators(df)

print(f'Всего признаков: {len(df.columns)}')
print(f'\nНовые признаки:')
new_cols = [c for c in df.columns if c not in ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'sector', 'is_earnings', 'is_dividend', 'event_impact']]
print(new_cols)

## 2. Лаговые Признаки

Добавляем прошлые значения как признаки для предсказания будущего.

In [ ]:
def add_lag_features(df, lags=[1, 2, 3, 5, 10]):
    """
    Добавляет лаговые признаки для ключевых переменных.
    """
    result_dfs = []
    
    for ticker in df['ticker'].unique():
        ticker_df = df[df['ticker'] == ticker].copy().sort_values('date')
        
        # Лаги для доходности
        ticker_df['return'] = ticker_df['close'].pct_change()
        for lag in lags:
            ticker_df[f'return_lag_{lag}'] = ticker_df['return'].shift(lag)
        
        # Лаги для объёма
        for lag in [1, 5]:
            ticker_df[f'volume_ratio_lag_{lag}'] = ticker_df['volume_ratio'].shift(lag)
        
        # Лаги для RSI
        for lag in [1, 5]:
            ticker_df[f'rsi_lag_{lag}'] = ticker_df['rsi'].shift(lag)
        
        # Лаги для волатильности
        ticker_df['volatility_lag_1'] = ticker_df['volatility_20'].shift(1)
        
        # Rolling статистики доходности
        ticker_df['return_mean_5'] = ticker_df['return'].rolling(5).mean()
        ticker_df['return_std_5'] = ticker_df['return'].rolling(5).std()
        ticker_df['return_mean_20'] = ticker_df['return'].rolling(20).mean()
        ticker_df['return_std_20'] = ticker_df['return'].rolling(20).std()
        
        # Streak (серия положительных/отрицательных дней)
        ticker_df['positive_return'] = (ticker_df['return'] > 0).astype(int)
        
        result_dfs.append(ticker_df)
    
    return pd.concat(result_dfs, ignore_index=True)

# Применяем
df = add_lag_features(df)

print(f'Всего признаков после лагов: {len(df.columns)}')

## 3. Целевые Переменные

Создаём несколько целевых переменных для разных задач:

- **Регрессия**: будущая доходность
- **Классификация**: направление движения
- **Multi-horizon**: прогноз на разные горизонты

In [ ]:
def add_target_variables(df):
    """
    Добавляет целевые переменные для предсказания.
    """
    result_dfs = []
    
    for ticker in df['ticker'].unique():
        ticker_df = df[df['ticker'] == ticker].copy().sort_values('date')
        
        # Будущие доходности (regression targets)
        ticker_df['target_return_1d'] = ticker_df['close'].shift(-1) / ticker_df['close'] - 1
        ticker_df['target_return_5d'] = ticker_df['close'].shift(-5) / ticker_df['close'] - 1
        ticker_df['target_return_10d'] = ticker_df['close'].shift(-10) / ticker_df['close'] - 1
        ticker_df['target_return_20d'] = ticker_df['close'].shift(-20) / ticker_df['close'] - 1
        
        # Направление (classification targets)
        ticker_df['target_direction_1d'] = (ticker_df['target_return_1d'] > 0).astype(int)
        ticker_df['target_direction_5d'] = (ticker_df['target_return_5d'] > 0).astype(int)
        
        # Значительное движение (>1%)
        ticker_df['target_big_move_up'] = (ticker_df['target_return_1d'] > 0.01).astype(int)
        ticker_df['target_big_move_down'] = (ticker_df['target_return_1d'] < -0.01).astype(int)
        
        # Будущая волатильность
        ticker_df['target_volatility_5d'] = ticker_df['return'].shift(-1).rolling(5).std().shift(-4) * np.sqrt(252)
        
        result_dfs.append(ticker_df)
    
    return pd.concat(result_dfs, ignore_index=True)

# Применяем
df = add_target_variables(df)

print('Целевые переменные добавлены:')
target_cols = [c for c in df.columns if c.startswith('target_')]
for col in target_cols:
    print(f'  {col}')

## 4. Exploratory Data Analysis

In [ ]:
# Базовая статистика признаков
feature_cols = [c for c in df.columns if c not in 
                ['date', 'ticker', 'sector', 'is_earnings', 'is_dividend'] 
                and not c.startswith('target_')]

print(f'Всего признаков для моделирования: {len(feature_cols)}')
print(f'\nСтатистика по ключевым признакам:')

key_features = ['close', 'return', 'rsi', 'macd', 'bb_position', 'atr_percent', 'volume_ratio']
df[key_features].describe().round(4)

In [ ]:
# Распределения ключевых признаков
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

features_to_plot = ['return', 'rsi', 'macd_hist', 'bb_position', 
                    'atr_percent', 'volume_ratio', 'roc_10', 'volatility_20']

for i, (ax, feat) in enumerate(zip(axes.flat, features_to_plot)):
    data = df[feat].dropna()
    ax.hist(data, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(data.mean(), color='red', linestyle='--', label=f'Mean: {data.mean():.3f}')
    ax.set_title(feat)
    ax.legend(fontsize=8)

plt.suptitle('Распределения ключевых признаков', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция признаков с целевой переменной
df_clean = df.dropna(subset=['target_return_1d'])

correlations = []
for col in feature_cols:
    if col in df_clean.columns:
        corr = df_clean[col].corr(df_clean['target_return_1d'])
        if not np.isnan(corr):
            correlations.append({'feature': col, 'correlation': corr})

corr_df = pd.DataFrame(correlations)
corr_df['abs_correlation'] = corr_df['correlation'].abs()
corr_df = corr_df.sort_values('abs_correlation', ascending=False)

# Визуализация топ-20
top_20 = corr_df.head(20)

fig, ax = plt.subplots(figsize=(10, 8))
colors = ['green' if c > 0 else 'red' for c in top_20['correlation']]
ax.barh(range(len(top_20)), top_20['correlation'], color=colors)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['feature'])
ax.set_xlabel('Корреляция с target_return_1d')
ax.set_title('Топ-20 признаков по корреляции с целевой переменной')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

print('\nТоп-10 признаков:')
print(corr_df[['feature', 'correlation']].head(10).to_string(index=False))

In [ ]:
# Корреляция между признаками (для выявления мультиколлинеарности)
selected_features = ['return', 'rsi', 'macd', 'macd_hist', 'bb_position', 'bb_width',
                    'atr_percent', 'volume_ratio', 'roc_10', 'volatility_20',
                    'momentum_10', 'price_sma20_ratio']

corr_matrix = df[selected_features].corr()

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            fmt='.2f', ax=ax)
ax.set_title('Корреляция между признаками')
plt.tight_layout()
plt.show()

# Выявляем сильные корреляции
print('\nСильно коррелированные пары (|r| > 0.7):')
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            print(f'  {corr_matrix.columns[i]} - {corr_matrix.columns[j]}: {corr_matrix.iloc[i, j]:.2f}')

In [ ]:
# Анализ целевой переменной
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Распределение доходности
returns = df['target_return_1d'].dropna()
axes[0, 0].hist(returns, bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0, color='red', linestyle='--')
axes[0, 0].set_title('Распределение 1-дневной доходности')
axes[0, 0].set_xlabel('Доходность')

# 2. QQ-plot
from scipy import stats
stats.probplot(returns, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('QQ-Plot (сравнение с нормальным распределением)')

# 3. Баланс классов
direction_counts = df['target_direction_1d'].value_counts()
axes[1, 0].bar(['Down (0)', 'Up (1)'], direction_counts.values)
for i, v in enumerate(direction_counts.values):
    axes[1, 0].text(i, v + 100, f'{v:,}\n({v/len(df)*100:.1f}%)', ha='center')
axes[1, 0].set_title('Баланс классов (направление)')
axes[1, 0].set_ylabel('Количество')

# 4. Автокорреляция доходностей
from pandas.plotting import autocorrelation_plot
sample_ticker = df[df['ticker'] == 'TECH_A']['return'].dropna()
autocorrelation_plot(sample_ticker, ax=axes[1, 1])
axes[1, 1].set_title('Автокорреляция доходностей (TECH_A)')
axes[1, 1].set_xlim(0, 50)

plt.tight_layout()
plt.show()

print(f'\nСтатистика целевой переменной:')
print(f'Mean: {returns.mean()*100:.4f}%')
print(f'Std: {returns.std()*100:.4f}%')
print(f'Skewness: {returns.skew():.4f}')
print(f'Kurtosis: {returns.kurtosis():.4f}')

## 5. Подготовка Данных для Моделирования

In [ ]:
# Удаляем строки с NaN в целевых переменных
df_clean = df.dropna(subset=['target_return_1d', 'target_direction_1d']).copy()

# Удаляем строки с NaN в важных признаках
important_features = ['rsi', 'macd', 'bb_position', 'atr_percent', 'sma_200']
df_clean = df_clean.dropna(subset=important_features)

print(f'Записей после очистки: {len(df_clean):,}')
print(f'Удалено: {len(df) - len(df_clean):,} ({(len(df) - len(df_clean))/len(df)*100:.1f}%)')

# Проверяем пропуски
missing = df_clean.isnull().sum()
missing_cols = missing[missing > 0]
if len(missing_cols) > 0:
    print(f'\nКолонки с пропусками:')
    print(missing_cols)
else:
    print('\nПропусков в данных нет')

In [ ]:
# Определяем наборы признаков для моделирования

# Все доступные признаки
all_features = [c for c in df_clean.columns if c not in 
                ['date', 'ticker', 'sector', 'is_earnings', 'is_dividend', 'event_impact']
                and not c.startswith('target_')]

# Базовый набор (основные индикаторы)
basic_features = [
    'open', 'high', 'low', 'close', 'volume',
    'return', 'rsi', 'macd', 'macd_hist', 'bb_position', 'atr_percent',
    'volume_ratio', 'volatility_20'
]

# Расширенный набор
extended_features = basic_features + [
    'sma_20', 'sma_50', 'ema_12', 'ema_26',
    'bb_width', 'roc_10', 'momentum_10',
    'price_sma20_ratio', 'price_sma50_ratio',
    'return_lag_1', 'return_lag_5', 'rsi_lag_1',
    'return_mean_5', 'return_std_5'
]

# Проверяем наличие всех признаков
missing_features = [f for f in extended_features if f not in df_clean.columns]
if missing_features:
    print(f'Отсутствующие признаки: {missing_features}')
    extended_features = [f for f in extended_features if f in df_clean.columns]

print(f'Всего признаков: {len(all_features)}')
print(f'Базовый набор: {len(basic_features)} признаков')
print(f'Расширенный набор: {len(extended_features)} признаков')

In [ ]:
# Сохраняем обработанные данные
df_clean.to_parquet(f'{data_dir}/processed_data.parquet', index=False)

# Сохраняем списки признаков
feature_sets = {
    'all_features': all_features,
    'basic_features': basic_features,
    'extended_features': extended_features
}

with open(f'{data_dir}/feature_sets.json', 'w') as f:
    json.dump(feature_sets, f, indent=2)

print('Данные сохранены:')
print(f'  {data_dir}/processed_data.parquet')
print(f'  {data_dir}/feature_sets.json')
print(f'\nРазмер processed_data: {os.path.getsize(f"{data_dir}/processed_data.parquet")/1024/1024:.2f} MB')

## Итоги

### Созданные признаки:

- **Трендовые**: SMA (5, 10, 20, 50, 200), EMA (12, 26, 50)
- **Моментум**: RSI, MACD, ROC, Momentum
- **Волатильность**: Bollinger Bands, ATR, Historical Volatility
- **Объём**: OBV, Volume Ratio
- **Лаговые**: прошлые доходности и индикаторы

### Целевые переменные:

- Доходность на 1, 5, 10, 20 дней
- Направление движения (binary)
- Значительные движения (>1%)

### Выводы EDA:

- Доходности имеют тяжёлые хвосты (высокий kurtosis)
- Низкая автокорреляция доходностей (эффективность рынка)
- Классы относительно сбалансированы (~50/50)
- Есть мультиколлинеарность между похожими индикаторами

### Следующий шаг:

В ноутбуке 03_classical_ml мы построим baseline модели:
- Logistic Regression
- Random Forest
- XGBoost / LightGBM